# WIP
## IES - Python - Project
### Marathon Results
### Data analysis
#### David Koubek, Jiri Zelenka

### Content
#### Import Required Packages
#### Data Loading + Merging
#### Technical Improvements:
##### - Renaming Columns + Adding Gender
##### - Cleaning Time + Adding Minutes
##### - Cleaning Names of Runners
-------------------------------------
#### Unique Countries, Names and Ages
#### Runners who Repeated Marathon
#### Percentage of Czechs Over Time
-------------------------------------
#### Pivot tables - Year x Gender
###### Min rank
###### Min time
###### Mean time
-------------------------------------
#### Medals
-------------------------------------
#### M/W Histograms over years

### Import required packages.

In [ ]:
import pandas as pd # for dataframe
import numpy as np  # working with arrays
import matplotlib.pyplot as plt  # Basic plots
import seaborn as sns   # Plots, boxplots

from collections import Counter

### Data loading + Merging

#### 1995 - 2019 (excluding 2012)

In [ ]:
years = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
         2005, 2006, 2007, 2008, 2009, 2010, 2011, # 2012 is missing !!!, 24 runs
         2013, 2014, 2015, 2016, 2017, 2018, 2019]

df_all = []

for i in years:
    df = pd.read_csv('../Scraper/Data_Marathons_Prague/data_' + str(i) + '.csv')
    df["Year"] = i
    df_all.append(df)
    
#  this creates a list of dfs, possible to get inside with indexes [1]...


df_merged = pd.concat(df_all, ignore_index=True)   # Important for obtaining overall index

len(df_merged)

In [ ]:
df_merged.head()
# df_merged.tail()

### Technical Improvements

#### Renaming Columns + Adding Gender
#### Cleaning Time + Adding Minutes
#### Cleaning Names of Runners

In [ ]:

### Renaming Columns

df_renamed = df_merged.rename(columns={"Age cat.":"Age", "Chip time":"Time","St. number":"Start_number"  })

## Adding Gender column  

df_renamed["Gender"] = df_renamed["Age"].astype(str).str[0]

# df_renamed.head()
df_renamed.tail()

In [ ]:

### Time Cleaning

    # Deleting 288 observations with time = "-" - mostly from year 1998, reason of this = ???
df_times = df_renamed[df_renamed.Time != "-"]

df_times.reset_index(drop = True,inplace=True)

    # Check for non-"-" times
times = df_times["Time"]
times.describe()

len(df_times)

# Keep working with 94 011 Runners = df_times


In [ ]:

### Counting minutes from Total time

tot_min = []
times = list(times)

for i in times:
    t=i.split(':')
    tot_min.append(int(t[0])*60+int(t[1])*1 +int(t[2])/60)
    
# len(tot_min)
tot_min = pd.DataFrame(tot_min)

df_mins = pd.concat([df_times, tot_min], axis=1)
df_mins = df_mins.rename(columns={0:"tot_mins"})


In [ ]:
    # List of roudned minutes

Mins = list(np.around(np.array(tot_min),0))
Mins = pd.DataFrame(Mins)

df_mins = pd.concat([df_mins, Mins], axis=1)
df_mins = df_mins.rename(columns={0:"Mins"})

df_mins.head()
# df_mins.tail()

# Keep working with Dataframe including rounded minutes = df_mins


In [ ]:

###  Cleaning Names:

all_names = list(df_mins.Name)

#print(sorted(all_names))

print(sorted(all_names)[0:500])
print(sorted(all_names)[len(all_names)-500:len(all_names)])

    # In the head we can see some names starting with spaces or dots - we will transform them.
    # In the tail there are names of Russian, Israeili, Chiense or Japanese Runners.
    # There are not so many of them, we keep them in original form


In [ ]:
indd = df_mins.index[df_mins['Name'] == ' Peter Puškár                   '].tolist()
df_mins.loc[indd,'Name'] = "Peter Puškár"
indd = df_mins.index[df_mins['Name'] == ' Radek Podracký               '].tolist()
df_mins.loc[indd,'Name'] = "Radek Podracký"
indd = df_mins.index[df_mins['Name'] == ' STEINIVAR'].tolist()
df_mins.loc[indd,'Name'] = "STEINIVAR"
indd = df_mins.index[df_mins['Name'] == ',Jiří Stránský Judr.'].tolist()
df_mins.loc[indd,'Name'] = "Jiří Stránský Judr."
indd = df_mins.index[df_mins['Name'] == '.Jiří Nekuža'].tolist()
df_mins.loc[indd,'Name'] = "Jiří Nekuža"
indd = df_mins.index[df_mins['Name'] == '- 50008873'].tolist()
df_mins.loc[indd,'Name'] = "- -"
indd = df_mins.index[df_mins['Name'] == '- 50044654'].tolist()
df_mins.loc[indd,'Name'] = "- -"
indd = df_mins.index[df_mins['Name'] == '?IVIND JOHANSEN'].tolist()
df_mins.loc[indd,'Name'] = "IVIND JOHANSEN"
indd = df_mins.index[df_mins['Name'] == '\u202aesty rosenberg\u202c\u200f'].tolist()
df_mins.loc[indd,'Name'] = "aesty rosenberg"
indd = df_mins.index[df_mins['Name'] == '\u202aron gashri\u202c\u200f'].tolist()
df_mins.loc[indd,'Name'] = "aron gashri"
indd = df_mins.index[df_mins['Name'] == 'ＨＩＲＯＫＯ ＫＵＳＵＮＯＫＩ'].tolist()
df_mins.loc[indd,'Name'] = "Hiroko Kusunoki"
indd = df_mins.index[df_mins['Name'] == 'ＨＩＲＯＫＯ ＳＨＩＮＯＭＩＹＡ'].tolist()
df_mins.loc[indd,'Name'] = "Hiroko Shinomiya"
indd = df_mins.index[df_mins['Name'] == 'ＨＩＲＯＭＩ ＹＯＮＥＺＵ'].tolist()
df_mins.loc[indd,'Name'] = "Hiromi Yonezu"
indd = df_mins.index[df_mins['Name'] == 'ＪＵＮＫＯ ＮＩＳＨＩＯＫＡ'].tolist()
df_mins.loc[indd,'Name'] = "Junko Nishioka"
indd = df_mins.index[df_mins['Name'] == 'ＫＩＳＡＫＯ ＴＯＫＯ'].tolist()
df_mins.loc[indd,'Name'] = "Kisako Toko"
indd = df_mins.index[df_mins['Name'] == 'ＫＩＹＯＫＯ ＫＡＷＡＳＡＫＩ'].tolist()
df_mins.loc[indd,'Name'] = "Kiyoko Kawasaki"
indd = df_mins.index[df_mins['Name'] == 'ＭＡＳＡＫＯ ＯＮＩＳＨＩ'].tolist()
df_mins.loc[indd,'Name'] = "Masako Onishi"
indd = df_mins.index[df_mins['Name'] == 'ＭＩＥＫＯ ＴＳＵＪＩＯＫＡ'].tolist()
df_mins.loc[indd,'Name'] = "Mieko Tsujioka"

In [ ]:

# Checking the names

all_names_new = list(df_mins.Name)

#print(sorted(all_names_new)[0:500])
print(sorted(all_names_new)[len(all_names_new)-500:len(all_names_new)])

In [ ]:
df_mins['Name'] = [str(i).lower() for i in df_mins['Name']] 
   # Changing all to lowercase
    
df_final = df_mins

In [ ]:
df_final.tail()

### Unique Countries, Names and Ages

In [ ]:
#All unique countries:

uq_countries = df_final.Nationality.unique()   
len(uq_countries)

# uq_countries

In [ ]:
# All unique ages:

uq_ages = df_final.Age.unique()
len(uq_ages)

# uq_ages

In [ ]:
# All unique names:

uq_names = df_final.Name.unique()
len(uq_names)

# uq_names

### Runners who Repeated Marathon

In [ ]:

############################################################################################x

# Count how many names are in multiple runs (not in the same run) and in how many:

all_names = list(df_final.Name)
num_of_runs = Counter(all_names)

num_of_runs.most_common()
    # Number of repeating persons.
    # However, yet we cant say which of them come from the same year ("Petr Svoboda"),
    # and who repeated several Runs ("jack joseph michel brossaud")

### Percentage of Czechs Over Time

In [ ]:
# Percentage of Czechs

# After inspecting each year, we noticed of the occurence of "TCH"
# instead of "CZE" in Nationality column.

# 1995 - no nationalities
# 1996 2x CZE + 0x TCH
# 1997 1x CZE + 727x TCH
# 1998 1x CZE + 1045x TCH
# 1999 986x CZE + 0x TCH
# ... CZE only


# Replacing all "TCH" with "CZE"

df_final['Nationality'] = df_final['Nationality'].replace('TCH', 'CZE')

In [ ]:
years = [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
         2005, 2006, 2007, 2008, 2009, 2010, 2011, 
         2013, 2014, 2015, 2016, 2017, 2018, 2019]

# 2012 is missing, 1995 and 1996 is excluded, 22 runs

In [ ]:
# Percents of czech Runners over years 1997-2019

cz_percent = []
for year in years:
    
    cz = len(df_final[(df_final["Nationality"] == "CZE") & (df_final["Year"] == year)])
    alls = len(df_final[df_final["Year"] == year])
    per = cz/alls
    cz_percent.append(per)
    
cz_percent

In [ ]:
czechs = df_final[df_final["Nationality"] == "CZE"]
len(czechs)/len(df_final)

# Overall percentage of czech Runners = 46.2 %


In [ ]:
plt.plot(years,cz_percent)
plt.show()

### Pivot tables 1/3

##### Gender and Years

##### -> Min + Mean rank
##### -> Min + Mean time


In [ ]:
# This table shows the top ranks of M and W of the ovreall ranks in year
# Here we can see that the top woman take place about 20 of all the Runners

df_genders = df_final[df_final["Gender"] != "-"]
pd.pivot_table(df_genders,index=["Gender"],columns = ["Year"], values="Rank",aggfunc=np.min)

In [ ]:
# This table presents average times of W + M over years
# Non clear Genders was excluded

df_genders = df_final[df_final["Gender"] != "-"]
pd.pivot_table(df_genders,index=["Gender"],columns = ["Year"], values="Mins",aggfunc=np.mean)


In [ ]:
# This table presents minimum times of W + M over years
# Non clear Genders was excluded

df_genders = df_final[df_final["Gender"] != "-"]
pd.pivot_table(df_genders,index=["Gender"],columns = ["Year"], values="Mins",aggfunc=np.min)


In [ ]:

## Let's plot Minimum and Mean times 1996-2019

df_man = df_final[df_final["Gender"] == "M"]
df_woman = df_final[df_final["Gender"] == "W"]

years = [1996,1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
         2005, 2006, 2007, 2008, 2009, 2010, 2011, 
         2013, 2014, 2015, 2016, 2017, 2018, 2019]

# 2012 is missing, 1995 is excluded because of no Genders, 23 runs


best_w = []

for i in years:
    new_w = df_woman[df_woman["Year"]== i]
    min_new_w = new_w.Mins.min()
    best_w.append(min_new_w)

best_m = []

for i in years:
    new_m = df_man[df_man["Year"]== i]
    min_new_m = new_m.Mins.min()
    best_m.append(min_new_m)
    
#-----------------------------------------
    
mean_w = []

for i in years:
    new_w = df_woman[df_woman["Year"]== i]
    mean_new_w = new_w.Mins.mean()
    mean_w.append(mean_new_w)

mean_m = []

for i in years:
    new_m = df_man[df_man["Year"]== i]
    mean_new_m = new_m.Mins.mean()
    mean_m.append(mean_new_m)
    

In [ ]:
# Average times evolution

plt.plot(years, mean_m)
plt.plot(years, mean_w)
plt.legend(['mean Man','mean Woman'], loc='upper left')
plt.show()

### Zásadní otázka, proč mám díru 2000 a naopak nemám díru 2012 ?
# Dá se to vyřešit opětovným reindexováním?

In [ ]:
# Top times evolution

plt.plot(years, best_m)
plt.plot(years, best_w)
plt.legend(['best Man','best Woman'], loc='upper right')
plt.show()

In [ ]:
sns.boxplot(x='Year', y='Mins', data=df_man)


In [ ]:
sns.boxplot(x='Year', y='Mins', data=df_woman)

In [ ]:
box_man = df_man['Mins']
box_woman = df_woman['Mins']
fig = plt.figure()
ax = fig.add_subplot(111)
ax.boxplot([box_man,box_woman], labels=['M', 'W'])

In [ ]:

sns.boxplot(x="Year", y="Mins",
            hue="Gender", palette=["m", "g"],
            data=df_genders)
sns.despine(offset=10, trim=True)

### Pivot tables 2/3

##### Nationality

##### -> Min + Mean rank
##### -> Min + Mean time


In [ ]:
df_nations = df_final[df_final["Nationality"] != "-"]
pd.pivot_table(df_nations,index=["Nationality"],columns = ["Year"], values="Rank",aggfunc=np.min)

In [ ]:

pd.pivot_table(df_nations,index=["Nationality"],columns = ["Year"], values="Mins",aggfunc=np.mean)


In [ ]:
pd.pivot_table(df_nations,index=["Nationality"],columns = ["Year"], values="Mins",aggfunc=np.min)


In [ ]:


########################################################################xx
uq_countries = uq_countries[uq_countries != "-"]
# uq_countries

national_means = []

for i in uq_countries:
        country = df_nations[df_nations["Nationality"]== i]
        country_mean = country.Mins.mean()
        national_means.append(country_mean)

print(national_means)

In [ ]:
plt.plot(national_means)
plt.show()

### Pivot tables 3/3

##### Age and Years

##### -> Min + Mean rank
##### -> Min + Mean time


In [ ]:
uq_countries

### Medals

In [ ]:
# Here we count number of Gold/Silver/Bronze Medals for each country, W/M

medals = uq_countries

counter_g = 0
counter_s = 0
coutner_b = 0

for abc in uq_countries:
    if [(df_nations["Rank"]==1) & (df_nations["Nationality"]== abc)]:
        counter_g = counter_g+1
    elif [(df_nations["Rank"]==2) & (df_nations["Nationality"]== abc)]:
        counter_s = counter_s+1
    elif [(df_nations["Rank"]==3) & (df_nations["Nationality"]== abc)]:
        counter_b = counter_b+1
    else:
    medals.append(counter_g, counter_s, counter_b)
    
    ########################################################################
    ##Nedaří se mi uložit medaile....